In [2]:
#----------------------------------------------
#This Notebook merges the cleaned data from FEMA and Zillow
#----------------------------------------------

import pandas as pd
import numpy as np
import os as os
import csv
import time

In [40]:
#Read and store the merged clean data
merged_clean_data = os.path.join("..","Cleaned Data","Merged_Clean_Data.csv")

merged_df = pd.read_csv(merged_clean_data)

merged_df = merged_df.drop(merged_df.columns[0], axis = 1)


merged_df.head()

,RegionName,CountyName,State,1996-05,1996-06,1996-07,1996-08,1996-09,1996-10,1996-11,...,Declaration Date,Fiscal Year Declared,Disaster Type,Incident Type,Title,Incident Begin Date,Incident End Date,Disaster Close Out Date,Declared County Area,Place Code
0,23464,Virginia Beach County,VA,-0.001936,-0.00194,0.0,0.000972,0.002913,0.003872,0.004822,...,1996-01,1996,DR,Snow,BLIZZARD OF 96 (SEVERE SNOW STORM),1996-01,1996-01,1997-11,Virginia Beach,82000.0
1,23464,Virginia Beach County,VA,-0.001936,-0.00194,0.0,0.000972,0.002913,0.003872,0.004822,...,1998-09,1998,DR,Hurricane,HURRICANE BONNIE,1998-08,1998-09,2005-05-20T00:00:00.000Z,Virginia Beach,82000.0
2,23464,Virginia Beach County,VA,-0.001936,-0.00194,0.0,0.000972,0.002913,0.003872,0.004822,...,1999-09,1999,DR,Hurricane,HURRICANE FLOYD MAJOR DISASTER DECLARATIONS,1999-09,1999-09,2007-03-29T00:00:00.000Z,Virginia Beach,82000.0
3,23464,Virginia Beach County,VA,-0.001936,-0.00194,0.0,0.000972,0.002913,0.003872,0.004822,...,2003-09,2003,DR,Hurricane,HURRICANE ISABEL,2003-09,2003-10,2013-09-25T23:59:00.000Z,Virginia Beach,82000.0
4,23464,Virginia Beach County,VA,-0.001936,-0.00194,0.0,0.000972,0.002913,0.003872,0.004822,...,2009-12,2010,DR,Severe Storm(s),SEVERE STORMS AND FLOODING ASSOCIATED WITH TRO...,2009-11,2009-11,2016-04-13T00:00:00.000Z,Virginia Beach,82000.0


In [61]:
t1 = []
t2 = []
t3 = []
t4 = []
t5 = []
t6 = []

for index, row in merged_df.iterrows():
    ddate = row['Declaration Date']
    try:
        col_idx = merged_df.columns.get_loc(ddate)
        t1.append(row.iloc[col_idx+1])
    except:
        t1.append(None)
    try:
        col_idx = merged_df.columns.get_loc(ddate)
        t2.append(row.iloc[col_idx+2])
    except:
        t2.append(None)
    try:
        col_idx = merged_df.columns.get_loc(ddate)
        t3.append(row.iloc[col_idx+3])
    except:
        t3.append(None)
    try:
        col_idx = merged_df.columns.get_loc(ddate)
        t4.append(row.iloc[col_idx+4])
    except:
        t4.append(None)
    try:
        col_idx = merged_df.columns.get_loc(ddate)
        t5.append(row.iloc[col_idx+5])
    except:
        t5.append(None) 
    try:
        col_idx = merged_df.columns.get_loc(ddate)
        t6.append(row.iloc[col_idx+6])
    except:
        t6.append(None)

merged_normalized_dict = {'title' : merged_df['Title'], 'disaster number' : merged_df['Disaster Number'],
                          'Type' : merged_df['Disaster Type'], 'Declaration Date' : merged_df['Declaration Date'],
                          'Zip Code' : merged_df['RegionName'], 'County' : merged_df['CountyName'],
                          't+1': t1, 't+2' : t2, 't+3': t3, 't+4' : t4, 't+5' : t5,
                          't+6' : t6}
merged_normalized_df = pd.DataFrame(merged_normalized_dict)

merged_normalized_df.head()

,title,disaster number,Type,Declaration Date,Zip Code,County,t+1,t+2,t+3,t+4,t5,t6
0,BLIZZARD OF 96 (SEVERE SNOW STORM),1086,DR,1996-01,23464,Virginia Beach County,NaN,NaN,NaN,NaN,NaN,NaN
1,HURRICANE BONNIE,1242,DR,1998-09,23464,Virginia Beach County,0.004472,0.003562,0.004437,0.004417,0.004398,0.004378
2,HURRICANE FLOYD MAJOR DISASTER DECLARATIONS,1293,DR,1999-09,23464,Virginia Beach County,0.004259,0.004241,0.004223,0.005046,0.003347,0.002502
3,HURRICANE ISABEL,1491,DR,2003-09,23464,Virginia Beach County,0.014539,0.015576,0.016564,0.017502,0.017794,0.019231
4,SEVERE STORMS AND FLOODING ASSOCIATED WITH TRO...,1862,DR,2009-12,23464,Virginia Beach County,0.000821,0.001230,-0.002048,-0.004514,-0.001237,-0.003302
